In [ ]:
# default_exp models.live_data

In [ ]:
import statsapi as mlb
import datetime as dt
from pydantic import (
    BaseModel,
    root_validator,
    validator,
    conint,
    conlist
)
from MLB_DataDevTools.models.base_models import *
from enum import Enum
from typing import Any,List,Optional

In [ ]:
import pandas as pd
from MLB_DataDevTools.database import create_mlb_engine
engine = create_mlb_engine()

In [ ]:
games = pd.read_sql('game',engine.connect())

pk = games['game_pk'].sample().iloc[0]

game = mlb.get('game',{'gamePk':pk})

liveData = game['liveData']

liveData.keys()

plays = liveData['plays']['allPlays']

play = plays[20]


## Plays

In [ ]:
play.keys()

dict_keys(['result', 'about', 'count', 'matchup', 'pitchIndex', 'actionIndex', 'runnerIndex', 'runners', 'playEvents', 'playEndTime', 'atBatIndex'])

In [ ]:
play['result']

{'type': 'atBat',
 'event': 'Walk',
 'eventType': 'walk',
 'description': 'Ronald Guzman walks.',
 'rbi': 0,
 'awayScore': 0,
 'homeScore': 0}

In [ ]:
event_types = mlb.get('meta',{'type':'eventTypes'})
event_types[0]

{'plateAppearance': False,
 'hit': False,
 'code': 'pickoff_1b',
 'baseRunningEvent': True,
 'description': 'Pickoff 1B'}

In [ ]:
class EventType(BaseModel):
    code: str
    plateAppearance: bool
    hit: bool
    baseRunningEvent: bool
    description: str
    
    @root_validator(pre=True)
    def get_event_type(cls,values):
        code = values['code']
        values.update(cls.Config.event_types[code])
        return values

    class Config:
        event_types = {
            pe.pop('code'):pe for pe in 
            mlb.get('meta',{'type':'eventTypes'})
        }
    def __init__(self,code):
        super().__init__(code=code)

In [ ]:
EventType(code='double')

EventType(code='double', plateAppearance=True, hit=True, baseRunningEvent=False, description='Double')

In [ ]:
EventType('double')

EventType(code='double', plateAppearance=True, hit=True, baseRunningEvent=False, description='Double')

In [ ]:
class ModelWithEventType(BaseModel):
    def __init__(self,eventType:str = None,**kwargs):
        super().__init__(
            eventType={'code':eventType},
            **kwargs
        )

In [ ]:
#exporti 

class PlayResult(ModelWithEventType):
    type: str
    event: str
    eventType: EventType
    description: str
    rbi: conint(ge=0)
    awayScore: conint(ge=0)
    homeScore: conint(ge=0)
    

In [ ]:
play['result']

{'type': 'atBat',
 'event': 'Walk',
 'eventType': 'walk',
 'description': 'Ronald Guzman walks.',
 'rbi': 0,
 'awayScore': 0,
 'homeScore': 0}

In [ ]:
PlayResult(**play['result'])

PlayResult(type='atBat', event='Walk', eventType=EventType(code='walk', plateAppearance=True, hit=False, baseRunningEvent=False, description='Walk'), description='Ronald Guzman walks.', rbi=0, awayScore=0, homeScore=0)

## Play About

In [ ]:
#exporti

class HalfInning(str,Enum):
    top='top'
    bottom='bottom'

class PlayAbout(BaseModel):
    atBatIndex: conint(ge=0)
    halfInning: HalfInning
    inning: conint(ge=0)
    startTime: dt.datetime
    endTime: dt.datetime
    isComplete: bool
    isScoringPlay: bool
    hasReview: bool
    hasOut: bool
    captivatingIndex: int

In [ ]:
play['about']

{'atBatIndex': 20,
 'halfInning': 'bottom',
 'isTopInning': False,
 'inning': 3,
 'startTime': '2020-09-26T23:50:00.443Z',
 'endTime': '2020-09-26T23:51:25.767Z',
 'isComplete': True,
 'isScoringPlay': False,
 'hasReview': False,
 'hasOut': False,
 'captivatingIndex': 0}

In [ ]:
PlayAbout(**play['about'])

PlayAbout(atBatIndex=20, halfInning=<HalfInning.bottom: 'bottom'>, inning=3, startTime=datetime.datetime(2020, 9, 26, 23, 50, 0, 443000, tzinfo=datetime.timezone.utc), endTime=datetime.datetime(2020, 9, 26, 23, 51, 25, 767000, tzinfo=datetime.timezone.utc), isComplete=True, isScoringPlay=False, hasReview=False, hasOut=False, captivatingIndex=0)

## Count

In [ ]:
play['count']

{'balls': 4, 'strikes': 1, 'outs': 0}

In [ ]:
#exporti 

class Count(BaseModel):
    balls: conint(ge=0,le=4)
    strikes: conint(ge=0,le=3)
    outs: conint(ge=0,le=3)

In [ ]:
Count(**play['count'])

Count(balls=4, strikes=1, outs=0)

## Matchup

In [ ]:
play['matchup']

{'batter': {'id': 608597,
  'fullName': 'Ronald Guzman',
  'link': '/api/v1/people/608597'},
 'batSide': {'code': 'L', 'description': 'Left'},
 'pitcher': {'id': 621121,
  'fullName': 'Lance McCullers Jr.',
  'link': '/api/v1/people/621121'},
 'pitchHand': {'code': 'R', 'description': 'Right'},
 'postOnFirst': {'id': 608597,
  'fullName': 'Ronald Guzman',
  'link': '/api/v1/people/608597'},
 'batterHotColdZones': [],
 'pitcherHotColdZones': [],
 'splits': {'batter': 'vs_RHP', 'pitcher': 'vs_LHB', 'menOnBase': 'Men_On'}}

In [ ]:
{p['matchup']['splits']['menOnBase'] for p in plays}

{'Empty', 'Men_On', 'RISP'}

In [ ]:
#exporti

class MenOnBase(str,Enum):
    Empty='Empty'
    Men_On='Men_On'
    RISP='RISP'

class BatterSplit(str,Enum):
    vs_RHP='vs_RHP'
    vs_LHP='vs_LHP'
class PitcherSplit(str,Enum):
    vs_RHB='vs_RHB'
    vs_LHB='vs_LHB'
class Splits(BaseModel):
    batter: BatterSplit
    pitcher: PitcherSplit
    menOnBase: MenOnBase

In [ ]:
#exporti

class Matchup(BaseModel):
    batter: PersonBase
    batSide: PlayerHandedness
    pitcher: PersonBase
    pitchHand: PlayerHandedness
    batterHotColdZones: conlist(Any,max_items=0) # I want to see if this ever comes back with items
    pitcherHotColdZones: conlist(Any,max_items=0)
    splits: Splits

In [ ]:
Matchup(**play['matchup'])

Matchup(batter=PersonBase(link=HttpUrl('https://statsapi.mlb.com/api/v1/people/608597', scheme='https', host='statsapi.mlb.com', tld='com', host_type='domain', path='/api/v1/people/608597'), id=608597, name='Ronald Guzman'), batSide=PlayerHandedness(code='L', description='Left'), pitcher=PersonBase(link=HttpUrl('https://statsapi.mlb.com/api/v1/people/621121', scheme='https', host='statsapi.mlb.com', tld='com', host_type='domain', path='/api/v1/people/621121'), id=621121, name='Lance McCullers Jr.'), pitchHand=PlayerHandedness(code='R', description='Right'), batterHotColdZones=[], pitcherHotColdZones=[], splits=Splits(batter=<BatterSplit.vs_RHP: 'vs_RHP'>, pitcher=<PitcherSplit.vs_LHB: 'vs_LHB'>, menOnBase=<MenOnBase.Men_On: 'Men_On'>))

In [ ]:
play['pitchIndex']

[0, 1, 2, 3, 4]

## Runners

In [ ]:
class Base(str,Enum):
    first_base='1B'
    second_base='2B'
    third_base='3B'
    score='score'

In [ ]:
plays[0]['about']['isScoringPlay']

False

In [ ]:
[p['runners'][0]['movement'] for p in plays if p['about']['isScoringPlay']][5:10]

[]

In [ ]:
play['runners'][0]['details']

{'event': 'Walk',
 'eventType': 'walk',
 'movementReason': None,
 'runner': {'id': 608597,
  'fullName': 'Ronald Guzman',
  'link': '/api/v1/people/608597'},
 'responsiblePitcher': None,
 'isScoringEvent': False,
 'rbi': False,
 'earned': False,
 'teamUnearned': False,
 'playIndex': 4}

In [ ]:
class Movement(BaseModel):
    originBase: Optional[Base] = None
    start: Optional[Base] = None
    end: Optional[Base] = None
    isOut: bool
    outNumber: Optional[conint(ge=0,le=3)] = None

In [ ]:
class RunnerDetails(ModelWithEventType):
    event: str
    eventType: EventType
    runner: PersonBase
    responsiblePitcher: Optional[PersonBase] = None
    isScoringEvent: bool
    rbi: bool
    earned: bool
    teamUnearned: bool
    playIndex: int

In [ ]:
play['runners'][0]['credits']

[]

In [ ]:
class Credit(BaseModel):
    player: PersonBase
    position: PositionBase
    credit: str

In [ ]:
play['runners'][0]

{'movement': {'originBase': None,
  'start': None,
  'end': '1B',
  'outBase': None,
  'isOut': False,
  'outNumber': None},
 'details': {'event': 'Walk',
  'eventType': 'walk',
  'movementReason': None,
  'runner': {'id': 608597,
   'fullName': 'Ronald Guzman',
   'link': '/api/v1/people/608597'},
  'responsiblePitcher': None,
  'isScoringEvent': False,
  'rbi': False,
  'earned': False,
  'teamUnearned': False,
  'playIndex': 4},
 'credits': []}

In [ ]:
class Runner(BaseModel):
    movement: Movement
    details: RunnerDetails
    credits: List[Credit]

In [ ]:
Runner(**play['runners'][0])

Runner(movement=Movement(originBase=None, start=None, end=<Base.first_base: '1B'>, isOut=False, outNumber=None), details=RunnerDetails(event='Walk', eventType=EventType(code='walk', plateAppearance=True, hit=False, baseRunningEvent=False, description='Walk'), runner=PersonBase(link=HttpUrl('https://statsapi.mlb.com/api/v1/people/608597', scheme='https', host='statsapi.mlb.com', tld='com', host_type='domain', path='/api/v1/people/608597'), id=608597, name='Ronald Guzman'), responsiblePitcher=None, isScoringEvent=False, rbi=False, earned=False, teamUnearned=False, playIndex=4), credits=[])

## Play Event

In [ ]:
play_event = play['playEvents'][0]

In [ ]:
play_event['details']

{'call': {'code': 'B', 'description': 'Ball'},
 'description': 'Ball',
 'code': 'B',
 'ballColor': 'rgba(39, 161, 39, 1.0)',
 'trailColor': 'rgba(50, 0, 221, 1.0)',
 'isInPlay': False,
 'isStrike': False,
 'isBall': True,
 'type': {'code': 'SI', 'description': 'Sinker'},
 'hasReview': False}

In [ ]:
play_event.keys()

dict_keys(['details', 'count', 'pitchData', 'index', 'playId', 'pitchNumber', 'startTime', 'endTime', 'isPitch', 'type'])

In [ ]:
play.keys()

dict_keys(['result', 'about', 'count', 'matchup', 'pitchIndex', 'actionIndex', 'runnerIndex', 'runners', 'playEvents', 'playEndTime', 'atBatIndex'])